<a href="https://colab.research.google.com/github/roberto0106/python_multi/blob/main/Envio_de_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Código para Google Colab: enviar relatórios agrupados via Gmail

import pandas as pd
import smtplib
from email.message import EmailMessage
from google.colab import files
import getpass

# 1. Faça upload do seu arquivo Excel
print("Selecione o arquivo Excel para upload:")
uploaded = files.upload()
aba = input("Digite o nome da aba: ")
# Pega o primeiro arquivo enviado
excel_path = next(iter(uploaded))
print(f"Arquivo carregado: {excel_path}\n")

# 2. Defina parâmetros de envio
email_column = 'email'  # Nome da coluna que contém os emails
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_user = input('Digite seu email Gmail (remetente): ')
smtp_password = getpass.getpass('Digite sua App Password do Gmail (senha): ')
subject_template = 'Relatório para {recipient}'
body_template = (
    'Olá, {recipient},!\n\n'
    'Bom dia, segue em anexo, extrato de sua premiação referente ao mes de Abril.\n\n'
    'Qualquer dúvida, estamos à disposição.'
)

# 3. Carrega e agrupa o DataFrame
print("Carregando planilha... ")
df = pd.read_excel(excel_path, sheet_name= aba)
if email_column not in df.columns:
    raise ValueError(f"Coluna de email '{email_column}' não encontrada no arquivo")
groups = df.groupby(email_column)
print(f"Encontrados {len(groups)} destinatários diferentes.\n")
df.head()

# 4. Envia um anexo Excel para cada grupo
print("Iniciando envios...")
for recipient, group in groups:
    safe_recipient = recipient.replace('@', '_at_').replace('.', '_')
    filename = f'report_{safe_recipient}.xlsx'
    group.to_excel(filename, index=False)

    # Monta a mensagem
    msg = EmailMessage()
    msg['Subject'] = subject_template.format(recipient=recipient)
    msg['From'] = smtp_user
    msg['To'] = recipient
    msg.set_content(body_template.format(recipient=recipient))

    # Anexa o arquivo Excel
    with open(filename, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(
        file_data,
        maintype='application',
        subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        filename=filename
    )

    # Envia via Gmail SMTP
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_user, smtp_password)
        server.send_message(msg)

    print(f"Enviado para {recipient}")

print("Todos os relatórios foram enviados!")


Selecione o arquivo Excel para upload:


Saving Premiações VPTECH 2025.xlsx to Premiações VPTECH 2025 (2).xlsx
Digite o nome da aba: Extrato para Envio Teste
Arquivo carregado: Premiações VPTECH 2025 (2).xlsx

Digite seu email Gmail (remetente): roberto.flor@grupomulti.com.br
Digite sua App Password do Gmail (senha): ··········
Carregando planilha... 
Encontrados 1 destinatários diferentes.

Iniciando envios...
Enviado para roberto.flor@grupomulti.com.br
Todos os relatórios foram enviados!
